In [1]:
import pandas as pd
import numpy as np
import pickle


The main objective of this nb is to create a seperate author_df, to be used with Scopus API Author Search

In [2]:
with open("q1_date_df", "rb") as fp:
    q1_date_df = pickle.load(fp)

In [8]:
q1_date_df.author.map(type).value_counts()

<class 'list'>     140744
<class 'float'>       249
Name: author, dtype: int64

It should be noted that 249 artcls do not have any author information :(

In [69]:
q1_author_df = q1_date_df.reset_index().dropna(subset= "author").loc[:,["DOI", "JRNL_ID", "author"]].explode("author")

In [70]:
q1_author_df.head()

,DOI,JRNL_ID,author
0,10.1002/aic.14056,1_0,"{'given': 'Phuong-Mai', 'family': 'Nguyen', 's..."
0,10.1002/aic.14056,1_0,"{'given': 'Audrey', 'family': 'Goujon', 'seque..."
0,10.1002/aic.14056,1_0,"{'given': 'Patrick', 'family': 'Sauvegrain', '..."
0,10.1002/aic.14056,1_0,"{'given': 'Olivier', 'family': 'Vitrac', 'sequ..."
1,10.1002/aic.14601,1_1,"{'given': 'Eiji', 'family': 'Iritani', 'sequen..."


In [76]:
q1_author_df.reset_index(drop=True, inplace=True)

In [72]:
aut_df = pd.DataFrame(list(q1_author_df['author']))

In [73]:
aut_df.head()

,given,family,sequence,ORCID,authenticated-orcid,suffix,name
0,Phuong-Mai,Nguyen,first,NaN,NaN,NaN,NaN
1,Audrey,Goujon,additional,NaN,NaN,NaN,NaN
2,Patrick,Sauvegrain,additional,NaN,NaN,NaN,NaN
3,Olivier,Vitrac,additional,NaN,NaN,NaN,NaN
4,Eiji,Iritani,first,NaN,NaN,NaN,NaN


In [78]:
q1_author_df = pd.concat([q1_author_df, aut_df], axis=1)

In [79]:
q1_author_df.head()

,DOI,JRNL_ID,author,given,family,sequence,ORCID,authenticated-orcid,suffix,name
0,10.1002/aic.14056,1_0,"{'given': 'Phuong-Mai', 'family': 'Nguyen', 's...",Phuong-Mai,Nguyen,first,NaN,NaN,NaN,NaN
1,10.1002/aic.14056,1_0,"{'given': 'Audrey', 'family': 'Goujon', 'seque...",Audrey,Goujon,additional,NaN,NaN,NaN,NaN
2,10.1002/aic.14056,1_0,"{'given': 'Patrick', 'family': 'Sauvegrain', '...",Patrick,Sauvegrain,additional,NaN,NaN,NaN,NaN
3,10.1002/aic.14056,1_0,"{'given': 'Olivier', 'family': 'Vitrac', 'sequ...",Olivier,Vitrac,additional,NaN,NaN,NaN,NaN
4,10.1002/aic.14601,1_1,"{'given': 'Eiji', 'family': 'Iritani', 'sequen...",Eiji,Iritani,first,NaN,NaN,NaN,NaN


Need to add Subj. Area colm. from jrnl_df for Scopus API:

In [80]:
with open("q1_jrnl_df", "rb") as fp:
    q1_jrnl_df = pickle.load(fp)

In [84]:
q1_jrnl_df.columns

Index(['ISSN', 'Year', 'Title', 'Sbj_Area', 'Total_Docs', 'Country', 'Region',
       'Publisher', 'Coverage', 'SJR', 'H index', 'Total Docs. (3years)',
       'Total Refs.', 'Total Cites (3years)', 'Citable Docs. (3years)',
       'Cites / Doc. (2years)', 'Ref. / Doc.', 'CR_Count', 'SH_Results',
       'Current_Count', 'Required_Count', 'Consider'],
      dtype='object')

In [92]:
q1_author_df = pd.merge(left=q1_author_df, right=q1_jrnl_df["Sbj_Area"], how="left", left_on="JRNL_ID", right_index=True)

In [97]:
q1_author_df["S_A"] = q1_author_df.Sbj_Area.map(lambda x: x.keys())

q1_author_df["S_A"] = q1_author_df.S_A.map(tuple)

In [112]:
# Keeping the original authır_df in case:

q1_author_expladed = q1_author_df.copy()

In [113]:
q1_author_df = q1_author_df.groupby(["given","family","S_A"])["DOI"].apply(list)

In [114]:
q1_author_df.head()

given         family     S_A                                             
& Richard E.  Tremblay   (Psychology, Social Sciences)                       [10.1016/j.jcrimjus.2016.02.013]
'Afa K.       Palu       (Medicine, Agricultural and Biological Sciences)                  [10.1002/pca.1246]
'Alim J.      Beveridge  (Business, Management and Accounting,)              [10.1016/j.jbusvent.2017.12.008]
(Joe) Yong    Zhou       (Business, Management and Accounting,)               [10.1080/10548408.2015.1075933]
(Luna) Jing   Cai        (Social Sciences, Arts and Humanities)                  [10.1016/j.jeap.2016.09.002]
Name: DOI, dtype: object

In [50]:
testo = q1_date_df.explode("author").author.tolist()

In [51]:
testo = [item for item in testo if type(item) == dict]

In [49]:
testo

[{'given': 'Phuong-Mai', 'family': 'Nguyen', 'sequence': 'first'},
 {'given': 'Audrey', 'family': 'Goujon', 'sequence': 'additional'},
 {'given': 'Patrick', 'family': 'Sauvegrain', 'sequence': 'additional'},
 {'given': 'Olivier', 'family': 'Vitrac', 'sequence': 'additional'},
 {'given': 'Eiji', 'family': 'Iritani', 'sequence': 'first'},
 {'given': 'Nobuyuki', 'family': 'Katagiri', 'sequence': 'additional'},
 {'given': 'Ryota', 'family': 'Nakajima', 'sequence': 'additional'},
 {'given': 'Kuo-Jen', 'family': 'Hwang', 'sequence': 'additional'},
 {'given': 'Tung-Wen', 'family': 'Cheng', 'sequence': 'additional'},
 {'given': 'Rachid', 'family': 'Chebbi', 'sequence': 'first'},
 {'ORCID': 'http://orcid.org/0000-0002-0924-4947',
  'authenticated-orcid': False,
  'given': 'Seyed Soheil',
  'family': 'Mansouri',
  'sequence': 'first'},
 {'given': 'Jakob Kjøbsted', 'family': 'Huusom', 'sequence': 'additional'},
 {'given': 'Rafiqul', 'family': 'Gani', 'sequence': 'additional'},
 {'given': 'Maurici